<h1>Problem 2</h1>
Cordonations of the Neighborhoods 

In [1]:
import sys
!{sys.executable} -m pip install geocoder

print('Packages installed.')

Packages installed.


In [2]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [5]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


In [6]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [7]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A\n,Not assigned\n,M2A\n,43.648690,-79.385440
1,M3A\n,North York\n,Parkwoods\n,43.752935,-79.335641
2,M4A\n,North York\n,Victoria Village\n,43.728102,-79.311890
3,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.650964,-79.353041
4,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.723265,-79.451211
5,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.661790,-79.389390
6,M8A\n,Not assigned\n,M9A\n,43.648690,-79.385440
7,M1B\n,Scarborough\n,"Malvern, Rouge\n",43.808626,-79.189913
8,M2B\n,Not assigned\n,M3B\n,43.648690,-79.385440
9,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens\n",43.707193,-79.311529


In [8]:
neighbors.shape

(135, 5)